In [23]:
import os
import re
import time
from datetime import datetime

import requests

from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [2]:
page_url = 'https://www.bexar.org/722/Jail-Activity-Reports'
xpath = '//a[text()="State and County"]'

In [32]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.get(page_url)
time.sleep(1)
link_elements = driver.find_elements_by_xpath(xpath)
links = [element.get_attribute('href') for element in link_elements]
driver.quit()

In [35]:
print(links)

['http://apps.bexar.org/jailactivity/KB50061D20190921.txt']


In [50]:
links = links[0:1]
for link in links:
    print(link)
    output_folder = os.path.expanduser('~/data/jailactivityreports')
    output_file_path = f'{output_folder}/{os.path.basename(link)}'
    report = requests.get(link).text
    line_start_pattern = re.compile('^0[A-Z]\d{9}')
    w_pattern = re.compile('W\d{7}')
    j_pattern = re.compile('J\d{7}')
    whitespace_pattern = re.compile('^\s{68}')
    lines = report.split('\r\n')
#     print(lines[:20])
    for i, line in enumerate(lines[:100], 1):
        if line_start_pattern.match(line):
#             print(f'{i}>>>{line}')
            booking_number = line[0:11]
            booking_date = datetime.strptime(line[14:29], '%m %d %Y %H%M')
            name = line[30:63].rstrip()
#             print(booking_number, booking_date, name)
        elif w_pattern.search(line) or j_pattern.search(line):
            try:
                action_date = datetime.strptime(line[10:25], '%m/%d/%Y %H%M')
            except:
                pass
            w_number = line[26:34].rstrip()
            j_number = line[35:43].rstrip()
            court= line[44:48]
            case_number = line[49:60].rstrip()
            charge = line[68:91]
            action = line[92:122]
            print(action_date, w_number, j_number, court, case_number, charge)
        elif whitespace_pattern.match(line):
            charge = line[68:91]
            action = line[92:122]
            print(charge)
        try:
            print(booking_number, booking_date, name, action_date, w_number, j_number, court, case_number, charge, action)
        except:
            pass
        
#             print(f'{i}>>>{line}')
#     with open(output_file_path, 'w') as output_file:
#         output_file.write(report)

http://apps.bexar.org/jailactivity/KB50061D20190921.txt
2019-09-20 19:17:00 W1653386 J1825976 D175 2018CR3060 FS PROMOTE PROSTITUTION
0B201852242 2018-10-31 09:37:00 WOODARD, DONALD E 2019-09-20 19:17:00 W1653386 J1825976 D175 2018CR3060 FS PROMOTE PROSTITUTION CASE DISMISSED    JD MOORE    
0B201852242 2018-10-31 09:37:00 WOODARD, DONALD E 2019-09-20 19:17:00 W1653386 J1825976 D175 2018CR3060 FS PROMOTE PROSTITUTION CASE DISMISSED    JD MOORE    
0B201906392 2019-02-08 08:51:00 OBERSTREET, LESTER 2019-09-20 19:17:00 W1653386 J1825976 D175 2018CR3060 FS PROMOTE PROSTITUTION CASE DISMISSED    JD MOORE    
2019-09-20 08:00:00  J1891822 D399 2019CR3919 F2 POSS W/I DEL CS PG1 
0B201906392 2019-02-08 08:51:00 OBERSTREET, LESTER 2019-09-20 08:00:00  J1891822 D399 2019CR3919 F2 POSS W/I DEL CS PG1  CASE DISMISSED    JD CASTRO   
2019-09-20 08:00:00  J1891824 D399 2019CR3848 FS EVADE ARREST/DETENTI
0B201906392 2019-02-08 08:51:00 OBERSTREET, LESTER 2019-09-20 08:00:00  J1891824 D399 2019CR3848